In [1]:
from datasets import load_dataset

ds = load_dataset("cnamuangtoun/resume-job-description-fit")

c:\Users\adedo\Desktop\Resume-Scanner\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\adedo\Desktop\Resume-Scanner\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adedo\.cache\huggingface\hub\datasets--cnamuangtoun--resume-job-description-fit. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an admi

In [19]:
ds['train'][120]

{'resume_text': "SummaryCoordinated Maintenance Electrician at a large flatline plastic company and printed product, that is skilled in repairing, testing, maintaining and installing electrical components. Proficient in reading blueprints and schematics. Tech savvy with ability to demonstrate work experience. Multiple years of experience in autocad programs and data driven enthusiast to analyze, diagnose the process of equipment to solve technical challenges and implement solutions. Constantly using data management process to improve and assist leadership skills to train my employees. Experienced supervisor, offering proven safety record, performance-oriented nature and strong history of accomplishment in a multitude of environments. Background training and mentoring team members to optimize performance and improve quality. Reliable, industrious and diplomatic in resolving concerns. Experience installing equipment and completing work in accordance with NEC codes. Committed job seeker w

In [38]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(example):
    # Tokenize separately
    tok = tokenizer(example['resume_text'], example['job_description_text'], truncation=True, padding="max_length", max_length=512, return_tensors="pt")

In [39]:
tokenized = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/6241 [00:00<?, ? examples/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returne

In [40]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [43]:
import evaluate

accuracy = evaluate.load("accuracy")

In [44]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [45]:
id2label = {0: "No Fit", 1: "Potential Fit", 2: "Good Fit"}
label2id = {"No Fit": 0, "Potential Fit": 1, "Good Fit": 2}

In [47]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()